Notes for myself: This is a revised version of Charging Demand and Charging Station Candidates.ipynb

In [76]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopy.distance import great_circle
from scipy.spatial import distance
from pulp import *

os.chdir('/Users/chengchen/Desktop/Insight/project/')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
df = pd.read_excel('data/raw/tts2016_ward_Toronto.xlsx')
df_trip = df[df.variable.str.contains('Number of trips made to the area as auto driver during ')]
df_trip['time_of_day'] = ['morning', 'afternoon']
df_trip = df_trip.drop('variable', axis = 1).transpose()
df_trip['ward'] = df_trip.index
df_trip.columns = ['morning trips', 'afternoon trips', 'ward']
df_trip.to_excel('data/processed/trip_to_wards.xlsx', index=False)


/Users/chengchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df_trip.head()

,morning trips,afternoon trips,ward
City of Toronto,639546,639519,City of Toronto
Ward 1,17318,11704,Ward 1
Ward 2,26302,16637,Ward 2
Ward 3,10726,16517,Ward 3
Ward 4,6783,15634,Ward 4


In [13]:
shp_path = 'data/raw/Toronto_wards/icitw_wgs84.shp'
df_ward = gpd.read_file(shp_path)
df_ward['ward'] = 'Ward '+df_ward['SCODE_NAME']
ward_trip_shp = df_ward.merge(df_trip, how='left')
#ward_trip_shp.plot(figsize=(10,10), column = 'morning trips',cmap='OrRd')

In [14]:
ward_trip_shp.head()

,GEO_ID,CREATE_ID,NAME,SCODE_NAME,LCODE_NAME,TYPE_DESC,TYPE_CODE,OBJECTID,SHAPE_AREA,SHAPE_LEN,geometry,ward,morning trips,afternoon trips
0,14630026,63519,Scarborough-Rouge River (41),41,EA41,Ward,CITW,1,0.0,0.0,POLYGON ((-79.26485565927024 43.77955621985134...,Ward 41,18476,16993
1,14630028,63519,Scarborough East (44),44,EA44,Ward,CITW,2,0.0,0.0,POLYGON ((-79.17076824694337 43.75563765091391...,Ward 44,9919,15938
2,14630024,63519,Scarborough-Rouge River (42),42,EA42,Ward,CITW,3,0.0,0.0,POLYGON ((-79.22568464642563 43.78940329829909...,Ward 42,18316,17767
3,14630027,63519,Scarborough-Agincourt (39),39,EA39,Ward,CITW,4,0.0,0.0,POLYGON ((-79.33141527345541 43.79311828976419...,Ward 39,10074,12000
4,14630035,63519,Willowdale (24),24,NO24,Ward,CITW,5,0.0,0.0,POLYGON ((-79.38719566413288 43.76348086985467...,Ward 24,19504,18917


In [15]:
# perhaps not useful
ward_trip_shp['ward centroid'] = ward_trip_shp['geometry'].centroid
ward_AM_trips = ward_trip_shp[['ward centroid','ward','morning trips']]
ward_AM_trips = gpd.GeoDataFrame(ward_AM_trips, geometry = 'ward centroid')
ward_AM_trips['long']=ward_AM_trips.centroid.x
ward_AM_trips['lat']=ward_AM_trips.centroid.y
ward_AM_trips.loc[ward_AM_trips['morning trips'].isnull(),'morning trips']=0

In [16]:
# CT shapefile
shp_path2 ='data/raw/Toronto_shp/Toronto_CMA_01_popn_age_sex_marital.shp'
df_TRT_shp = gpd.read_file(shp_path2)
ward_trip_shp.crs = {'init' :'epsg:4326'}
df_TRT_shp.crs == ward_trip_shp.crs
ward_trip_shp = ward_trip_shp.to_crs(df_TRT_shp.crs)
CT_trips = gpd.sjoin(df_TRT_shp, ward_trip_shp, how="inner", op="intersects")


In [17]:
CT_trips.head()
# each row represents a census tract, the info of "# trips in the ward that this CT belongs to" is added

,CTNAME,GEOGRAPHY,CTUID,CMAUID,PRUID,POP01,POP06,POPCHG,L_AREA,POP06RD,...,LCODE_NAME,TYPE_DESC,TYPE_CODE,OBJECTID,SHAPE_AREA,SHAPE_LEN,ward,morning trips,afternoon trips,ward centroid
0,0001.00,0001.00 (535000100) 010,5350001.00,535,35,626,571,-8.8,6.10,570,...,SO28,Ward,CITW,39,0.0,0.0,Ward 28,28180,13146,POINT (-79.37207815515001 43.63908140666023)
1,0002.00,0002.00 (535000200) 000,5350002.00,535,35,658,627,-4.7,3.17,625,...,SO28,Ward,CITW,39,0.0,0.0,Ward 28,28180,13146,POINT (-79.37207815515001 43.63908140666023)
13,0012.00,0012.00 (535001200) 000,5350012.00,535,35,2995,8053,168.9,1.11,8055,...,SO28,Ward,CITW,39,0.0,0.0,Ward 28,28180,13146,POINT (-79.37207815515001 43.63908140666023)
14,0013.00,0013.00 (535001300) 010,5350013.00,535,35,6365,6315,-0.8,0.76,6315,...,SO28,Ward,CITW,39,0.0,0.0,Ward 28,28180,13146,POINT (-79.37207815515001 43.63908140666023)
15,0014.00,0014.00 (535001400) 000,5350014.00,535,35,516,548,6.2,0.47,545,...,SO28,Ward,CITW,39,0.0,0.0,Ward 28,28180,13146,POINT (-79.37207815515001 43.63908140666023)


In [18]:
CT_trips.shape # some CTs might intersect with 

(1046, 121)

In [28]:
CT_trip_ward27 = CT_trips.loc[CT_trips['ward']=='Ward 27']
CT_trip_ward27.shape # 30 CTs intersecting with Ward 27

(30, 121)

In [29]:
# get the CTs' centroids
CT_trip_ward27['CT_centroid'] = CT_trip_ward27['geometry'].centroid
CT_trip_ward27.head()

/Users/chengchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,CTNAME,GEOGRAPHY,CTUID,CMAUID,PRUID,POP01,POP06,POPCHG,L_AREA,POP06RD,...,TYPE_DESC,TYPE_CODE,OBJECTID,SHAPE_AREA,SHAPE_LEN,ward,morning trips,afternoon trips,ward centroid,CT_centroid
15,0014.00,0014.00 (535001400) 000,5350014.00,535,35,516,548,6.2,0.47,545,...,Ward,CITW,35,0.0,0.0,Ward 27,29924,14980,POINT (-79.38071683425223 43.67385516042321),POINT (-79.38210130618089 43.64861246762182)
16,0015.00,0015.00 (535001500) 010,5350015.00,535,35,2468,2742,11.1,0.28,2740,...,Ward,CITW,35,0.0,0.0,Ward 27,29924,14980,POINT (-79.38071683425223 43.67385516042321),POINT (-79.37532957743522 43.65063174579927)
17,0016.00,0016.00 (535001600) 020,5350016.00,535,35,2532,4484,77.1,0.67,4480,...,Ward,CITW,35,0.0,0.0,Ward 27,29924,14980,POINT (-79.38071683425223 43.67385516042321),POINT (-79.36362004921544 43.65382275796478)
33,0032.00,0032.00 (535003200) 020,5350032.00,535,35,5413,5469,1.0,0.42,5465,...,Ward,CITW,35,0.0,0.0,Ward 27,29924,14980,POINT (-79.38071683425223 43.67385516042321),POINT (-79.37136519396184 43.66088716490177)
34,0033.00,0033.00 (535003300) 020,5350033.00,535,35,5143,5528,7.5,0.32,5530,...,Ward,CITW,35,0.0,0.0,Ward 27,29924,14980,POINT (-79.38071683425223 43.67385516042321),POINT (-79.36948332375061 43.65658854155352)


In [30]:
desired_columns = ['CT_centroid', 'morning trips', 'CTNAME']
CT_trip_ward27_clean =CT_trip_ward27[desired_columns] 

In [31]:
CT_trip_ward27_clean['morning trips to CT'] = CT_trip_ward27_clean['morning trips']/30
CT_trip_ward27_clean # 30 demand points

/Users/chengchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,CT_centroid,morning trips,CTNAME,morning trips to CT
15,POINT (-79.38210130618089 43.64861246762182),29924,0014.00,997.467
16,POINT (-79.37532957743522 43.65063174579927),29924,0015.00,997.467
17,POINT (-79.36362004921544 43.65382275796478),29924,0016.00,997.467
33,POINT (-79.37136519396184 43.66088716490177),29924,0032.00,997.467
34,POINT (-79.36948332375061 43.65658854155352),29924,0033.00,997.467
35,POINT (-79.37803247267108 43.65748620471367),29924,0034.00,997.467
36,POINT (-79.3849106674067 43.65612577587146),29924,0035.00,997.467
64,POINT (-79.39742348875959 43.66324600129045),29924,0061.00,997.467
65,POINT (-79.38722292786753 43.66793842532085),29924,0062.01,997.467
66,POINT (-79.38924818019258 43.66436153533483),29924,0062.02,997.467


In [34]:
# clean the demand dataset
CT_trip_ward27_clean = gpd.GeoDataFrame(CT_trip_ward27_clean, geometry = 'CT_centroid')
CT_trip_ward27_clean['long']=CT_trip_ward27_clean.CT_centroid.x
CT_trip_ward27_clean['lat']=CT_trip_ward27_clean.CT_centroid.y
CT_trip_ward27_clean.head()

,CT_centroid,morning trips,CTNAME,morning trips to CT,long,lat
15,POINT (-79.38210130618089 43.64861246762182),29924,0014.00,997.467,-79.382101,43.648612
16,POINT (-79.37532957743522 43.65063174579927),29924,0015.00,997.467,-79.375330,43.650632
17,POINT (-79.36362004921544 43.65382275796478),29924,0016.00,997.467,-79.363620,43.653823
33,POINT (-79.37136519396184 43.66088716490177),29924,0032.00,997.467,-79.371365,43.660887
34,POINT (-79.36948332375061 43.65658854155352),29924,0033.00,997.467,-79.369483,43.656589


In [37]:
# set of 
demand_CT_ward27 = CT_trip_ward27_clean[['CTNAME','morning trips to CT','long','lat']].set_index('CTNAME')

In [46]:
def gen_demand(df, column, EVPR, home_chg_ratio, long_commute_ratio):
    """generate the demand for charging (during day time) for each census tract"""
    pr_ch = long_commute_ratio * 1 + (1-long_commute_ratio)*(1-home_chg_ratio)/5
    # assumptions here: (1) ppl with longer commute (from outside of the city) always need public charging
    #                   (2) ppl with short commute only need public charging (1/5 of the times) if they do not have access to home charging
    df['demand_chg'] = df[column]*EVPR*pr_ch
    df = df.round({'demand_chg':0})
    return df
EVPR = 0.01
home_chg_ratio = 0.8
long_commute_ratio = 0.1
df_demand = gen_demand(demand_CT_ward27, 'morning trips to CT', EVPR, home_chg_ratio, long_commute_ratio)
# round the demand_chg to be integer so that it's easier to calculate
# need to modify the code so that it rounds automatically
df_demand['demand_chg'] = 1
df_demand

,morning trips to CT,long,lat,demand_chg
CTNAME,,,,
0014.00,997.467,-79.382101,43.648612,1
0015.00,997.467,-79.375330,43.650632,1
0016.00,997.467,-79.363620,43.653823,1
0032.00,997.467,-79.371365,43.660887,1
0033.00,997.467,-79.369483,43.656589,1
0034.00,997.467,-79.378032,43.657486,1
0035.00,997.467,-79.384911,43.656126,1
0061.00,997.467,-79.397423,43.663246,1
0062.01,997.467,-79.387223,43.667938,1


In [48]:
# Next step, spacial join parking lot with ward shape and get the parking lots in ward 27
df_parking = pd.read_excel('data/raw/TRT_parking.xlsx')
def df_to_gdf(df):
    """takes a dataframe with columns named 'longitude' and 'latitude' 
    to transform to a geodataframe with point features"""
    
    df['coordinates'] = df[['longitude', 'latitude']].values.tolist()
    df['coordinates'] = df['coordinates'].apply(Point)
    df = gpd.GeoDataFrame(df, geometry = 'coordinates')
    return df
gdf_parking = df_to_gdf(df_parking)
gdf_parking.head()

,ID,Name,latitude,longitude,Rating,Url,coordinates
0,ChIJaQRol2FHK4gRh8JJsjatDhI,Impark Parking (Mississauga Executive Centre P...,43.597325,-79.635916,4.1,https://maps.google.com/?cid=1301167792787669639,POINT (-79.6359159 43.5973255)
1,ChIJo8d888s1K4gRX3ei2idCu68,Impark Parking,43.577757,-79.616419,4.0,https://maps.google.com/?cid=12662787516221519711,POINT (-79.6164191 43.57775729999999)
2,ChIJ6XIlmO5HK4gRUfdXjbN0A_E,Impark (Sussex Centre),43.591805,-79.636268,0.0,https://maps.google.com/?cid=17366852902590084945,POINT (-79.636268 43.5918047)
3,ChIJMU1fXWlHK4gR-SWjLZB_xGs,Mississauga Civic Centre Underground Parking Lot,43.589077,-79.643238,4.0,https://maps.google.com/?cid=7765471914711262713,POINT (-79.64323779999999 43.58907720000001)
4,ChIJk5WzCBlHK4gRukvEsz5RM48,Impark (Parking),43.603299,-79.649067,0.0,https://maps.google.com/?cid=10318680500981746618,POINT (-79.6490666 43.6032989)


In [57]:
gdf_parking.crs = {'init' :'epsg:4326'}
gdf_parking.crs == df_ward.crs
parking_ward = gpd.sjoin(gdf_parking, df_ward, how="inner", op="within")
parking_ward.head()

,ID,Name,latitude,longitude,Rating,Url,coordinates,index_right,GEO_ID,CREATE_ID,NAME,SCODE_NAME,LCODE_NAME,TYPE_DESC,TYPE_CODE,OBJECTID,SHAPE_AREA,SHAPE_LEN,ward
55,ChIJ5SiFHaDR1IkRqjHXbUB6zOc,McDairmid Woods Park,43.780522,-79.266924,3.9,https://maps.google.com/?cid=16702859535149642154,POINT (-79.266924 43.780522),0,14630026,63519,Scarborough-Rouge River (41),41,EA41,Ward,CITW,1,0.0,0.0,Ward 41
57,ChIJuRi7_XDR1IkRM5X4RXTfm8g,Agincourt Recreation Centre,43.788048,-79.276242,3.8,https://maps.google.com/?cid=14455393119458858291,POINT (-79.2762419 43.7880479),0,14630026,63519,Scarborough-Rouge River (41),41,EA41,Ward,CITW,1,0.0,0.0,Ward 41
57,ChIJofioV2vR1IkR143Hl-jNgSE,Commander Park Arena,43.795021,-79.267795,3.3,https://maps.google.com/?cid=2414437274109840855,POINT (-79.2677951 43.7950207),0,14630026,63519,Scarborough-Rouge River (41),41,EA41,Ward,CITW,1,0.0,0.0,Ward 41
4,ChIJr2JjwKLb1IkROwSeRxITT8E,Port Union Village Common Park,43.776451,-79.134744,4.5,https://maps.google.com/?cid=13929373141712110651,POINT (-79.1347442 43.7764512),1,14630028,63519,Scarborough East (44),44,EA44,Ward,CITW,2,0.0,0.0,Ward 44
15,ChIJ-0Owhbzb1IkR1762gMyzNqA,Lawrence Ave East At Port Union Rd West Side,43.779533,-79.137367,3.0,https://maps.google.com/?cid=11544612385725005527,POINT (-79.137367 43.779533),1,14630028,63519,Scarborough East (44),44,EA44,Ward,CITW,2,0.0,0.0,Ward 44


In [67]:
parking_ward27 = parking_ward[parking_ward['ward']=='Ward 27'][['latitude','longitude','Rating','ID']]

In [104]:
chg_lc_27 = parking_ward27[parking_ward27['Rating']>3]
chg_lc_27.shape

(37, 4)

In [120]:
# try optimization: 30 demand points, 26 charging station location candidates

In [105]:
df_chg = chg_lc_27
demand_lc = df_demand.index.tolist()
len(demand_lc)

30

In [106]:
chg_lc = df_chg.index.tolist()
len(chg_lc)

37

In [107]:
df_demand.head()

,morning trips to CT,long,lat,demand_chg
CTNAME,,,,
0014.00,997.467,-79.382101,43.648612,1
0015.00,997.467,-79.375330,43.650632,1
0016.00,997.467,-79.363620,43.653823,1
0032.00,997.467,-79.371365,43.660887,1
0033.00,997.467,-79.369483,43.656589,1


In [108]:
df_demand.demand_chg.sum()

30

In [109]:
demand = df_demand['demand_chg'].to_dict()

In [110]:
df_chg['fixed_cost'] = 11000
fixed_cost = df_chg['fixed_cost'].to_dict()

/Users/chengchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [111]:
df_chg['chg_capacity'] = 10
capacity = df_chg['chg_capacity'].to_dict()

/Users/chengchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [112]:
# distance matrix
coords_pk = [(x,y) for x,y in zip(df_chg['longitude'],df_chg['latitude'])]
coords_trip = [(x,y) for x,y in zip(df_demand['long'],df_demand['lat'])]

distance_matrix = distance.cdist(coords_pk, coords_trip, 'euclidean')
transfer_ratio = 85
distance_matrix2 = transfer_ratio*distance_matrix

In [113]:
df_distance = pd.DataFrame(distance_matrix2, index = df_chg.index.tolist() ,columns = df_demand.index.tolist())

In [114]:
df_distance.head()

,0014.00,0015.00,0016.00,0032.00,0033.00,0034.00,0035.00,0061.00,0062.01,0062.02,...,0088.00,0089.00,0090.00,0091.01,0091.02,0120.00,0124.00,0125.00,0126.00,0186.00
51,1.197170,1.185425,1.759916,0.944399,1.208875,0.573863,0.599997,1.283997,0.609734,0.605057,...,0.850984,1.136795,1.492398,1.646584,1.945298,1.942611,2.153792,2.320665,3.018298,3.049851
46,1.342744,1.253260,1.707812,0.848943,1.161740,0.626969,0.784079,1.421775,0.630335,0.723774,...,0.749610,1.155525,1.471862,1.724772,1.994017,1.916030,2.070370,2.177309,2.882489,2.860658
25,1.421471,1.572841,2.237244,1.422830,1.686508,1.010927,0.739780,0.828402,0.307674,0.128111,...,0.772026,0.736605,1.142965,1.169458,1.486204,1.588402,1.915025,2.251040,2.908056,3.192103
14,1.627360,1.647004,2.141383,1.270136,1.597253,1.029301,0.973480,1.140279,0.214383,0.471594,...,0.431401,0.722158,1.040398,1.328783,1.570131,1.488049,1.696799,1.929222,2.608554,2.818549
2,1.736005,1.880692,2.490771,1.641375,1.940663,1.300132,1.054245,0.775454,0.190248,0.281725,...,0.630156,0.420454,0.828212,0.935101,1.208493,1.272017,1.622372,2.026665,2.656927,3.075005


In [115]:
df_travel_cost = df_distance * 2 
dic_cost_matrix = df_travel_cost.to_dict('index')

In [116]:
prob = LpProblem('FacilityLocation', LpMinimize)
serv_vars = LpVariable.dicts("Service",
                             [(i,j) for i in demand_lc
                                    for j in chg_lc],
                              0)
use_vars = LpVariable.dicts("UseLocation", chg_lc, 0, 1, LpBinary)
prob += lpSum(fixed_cost[j]*use_vars[j] for j in chg_lc) + lpSum(dic_cost_matrix[j][i]*serv_vars[(i,j)] for j in chg_lc for i in demand_lc)
for i in demand_lc:
    prob += lpSum(serv_vars[(i,j)] for j in chg_lc) == demand[i] # constraint 1
for j in chg_lc:
    prob += lpSum(serv_vars[(i,j)] for i in demand_lc) <= capacity[j]*use_vars[j]
for i in demand_lc:
    for j in chg_lc:
        prob += serv_vars[(i,j)] <= demand[i]*use_vars[j]

In [117]:
prob.solve()
print("Status: ", LpStatus[prob.status])

Status:  Optimal


In [119]:
TOL = .00001
for i in chg_lc:
    if use_vars[i].varValue > TOL:
        print("Eslablish charging station at site", i)

#for v in prob.variables():
#    print(v.name, "=", v.varValue)
    
# OPTIMAL SOLUTION
print("Total social cost is", value(prob.objective))

Eslablish charging station at site 9
Eslablish charging station at site 54
Eslablish charging station at site 50
Total social cost is 33053.8099889265
